                                                          Word2Vec_LR

In [1]:
import re
import string
import pandas as pd
import nltk
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

# Download NLTK stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Load dataset
df = pd.read_csv("data.csv")

# Apply text preprocessing
df["text_clean"] = df["text"].str.lower()
df["text_clean"] = df["text_clean"].str.replace(r'\d+', '', regex=True)
df["text_clean"] = df["text_clean"].str.translate(str.maketrans('', '', string.punctuation))
df["text_clean"] = df["text_clean"].apply(lambda text: ' '.join([word for word in text.split() if word not in stop_words]))

# Tokenize text for Word2Vec
sentences = [text.split() for text in df["text_clean"]]

# Train Word2Vec model
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=2, workers=4)

# Function to get sentence vectors
def get_sentence_vector(words, model):
    vectors = [model.wv[word] for word in words if word in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

# Convert text data to Word2Vec feature vectors
X_vectors = np.array([get_sentence_vector(text.split(), word2vec_model) for text in df["text_clean"]])

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_vectors, df["label"], test_size=0.2, random_state=42)

# Train Logistic Regression Model
model = LogisticRegression(max_iter=500)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.4f}")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hano\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Model Accuracy: 0.9683


In [3]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, learning_curve


In [5]:
print(X_vectors.shape)
print(y_train.shape)

(39942, 100)
(31953,)


In [6]:
X_vectors =X_vectors[:len(y_train)]

In [11]:
import json

accuracy = 0.9683
with open("accuracy_Word2vector_LR.json", "w") as f:
    json.dump({"accuracy": accuracy}, f)